# Evaluating Numerical Expressions

In this project, we'll use stacks to build an function that can evaluate complex numerical expressions stored within a string.

For example, to calculate the expression `21 / (11 - 4) * 2 + 5`, we'll implement a function named `evaluate()` to evaluate the expression stored as a string.

    expression = "21 / (11 - 4) * 2 + 5"
    print(evaluate(expression))
    11

First we'll define a class called `Stack` that can be used to...

In [1]:
class Node:
    
    def __init__(self, data):
        self.data = data
        self.prev = None
        self.next = None

In [2]:
class Stack:

    def __init__(self):
        self.head = None
        self.tail = None
        self.length = 0

    def append(self, data):
        new_node = Node(data)
        if self.length == 0:
            self.head = self.tail = new_node
        else:
            self.tail.next = new_node
            new_node.prev = self.tail
            self.tail = new_node
        self.length += 1

    def __iter__(self):
        self._iter_node = self.head
        return self 

    def __next__(self):
        if self._iter_node is None:
            raise StopIteration
        ret = self._iter_node.data
        self._iter_node = self._iter_node.next
        return ret

    def prepend(self, data):
        new_node = Node(data)
        if self.length == 0:
            self.head = self.tail = new_node
        else:
            self.head.prev = new_node
            new_node.next = self.head
            self.head = new_node
        self.length += 1

    def __len__(self):
        return self.length

    def __str__(self):
        return str([value for value in self])
    
    def push(self, data):
        self.append(data)

    def peek(self):
        return self.tail.data

    def pop(self):
        ret = self.tail.data
        if self.length == 1:
            self.tail = self.head = None
        else:
            self.tail = self.tail.prev
            self.tail.next = None
        self.length -= 1
        return ret

## Infix & Postfix Notation


In [3]:
def tokenize(expression):
    return expression.split()

print(tokenize("12 2 4 + / 21 *"))

['12', '2', '4', '+', '/', '21', '*']


## Processing an Operator

In [4]:
def process_minus(stack):
    top = stack.pop()
    second_to_top = stack.pop()
    result = second_to_top - top  # subtraction
    stack.push(result)

In [5]:
def process_plus(stack):
    top = stack.pop()
    second_to_top = stack.pop()
    result = second_to_top + top # addition
    stack.push(result)

In [6]:
def process_time(stack):
    top = stack.pop()
    second_to_top = stack.pop()
    result = second_to_top * top # multiplication
    stack.push(result)

In [7]:
def process_divide(stack):
    top = stack.pop()
    second_to_top = stack.pop()
    result = second_to_top / top # division
    stack.push(result)

In [8]:
def process_pow(stack):
    top = stack.pop()
    second_to_top = stack.pop()
    result = second_to_top ** top # exponential
    stack.push(result)

## Evaluating Postfix Expressions

Next, we'll create a function called `evaluate_postfix()` in order to tokenize our expression. This way we can extract the operator and call the corresponding function to process the expression. 

In [9]:
def evaluate_postfix(expression):
    tokens = tokenize(expression)
    stack = Stack()
    for token in tokens:
        if token == '-':
            process_minus(stack)
        elif token == '+':
            process_plus(stack)
        elif token == '*':
            process_time(stack)
        elif token == '/':
            process_divide(stack)
        elif token == '**':
            process_pow(stack)
        else:
            stack.push(float(token))
    return stack.pop()

In [11]:
# Testing the function
expressions = [
    "4 6 -",
    "4 1 2 9 3 / * + 5 - *",
    "1 2 + 3 -",
    "1 2 - 3 +",
    "10 3 5 * 16 4 - / +",
    "5 3 4 2 - ** *",
    "12 2 4 + / 21 *",
    "1 1 + 2 **",
    "1 1 2 ** +"
]

for expression in expressions:
    print(evaluate_postfix(expression))

-2.0
8.0
0.0
2.0
11.25
45.0
42.0
4.0
2.0


## Operator Precedence in Infix Notation

## Handling Closing Parenthesis

## Handling Operators

## Handling Numbers